# Préparation des figures de statistiques descriptives

### Todo : 
Produire les figures, en global et par institution :
1. Ratio Femme/reste pour les acquisitions par année depuis 1945 : **DONE**
2. Age moyen à l'acquisition des hommes et des femmes (abandonné : deux bars a cote, mettre aussi le Q1 et Q3): **DONE**
3. Ratio oeuvres d'artistes Français/reste dans les acquisitions par nationalité (une courbe pour les hommes, une pour les femmes)
4. Sortir sous forme de matrice colorée le ratio H/F dans les 29 domaines en fonction du temps
5. Sous forme de courbe, une par mode d'acquisition, le ratio de femme

### Important pour le rapport :
- sortir les figures en ratio 2:1
- format .png, background transparent

In [87]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import re
import unicodedata

In [196]:
plt.rcParams["font.family"] = 'Tahoma'
pd.set_option('display.max_columns',200)
pd.set_option('display.max_rows',200)
global color_h, color_f, color_b
color_h = "lightblue"
color_f = "darkblue"
color_b = "lightgray"

In [4]:
# Les données sont stockées dans le dossier "data", c'est plus clean
authors  = pd.read_csv('../data/ALL_AUTHORS.csv',sep=';', low_memory=False)
art = pd.read_csv('../data/ALL_ARTWORKS.csv',low_memory=False)

### Etape préliminaire 1 : "donner un genre" aux oeuvres

In [5]:
authors['Gender'].value_counts()

masculin         24914
féminin           7170
non renseigné     1638
groupe             666
Name: Gender, dtype: int64

In [6]:
art['Gender'] = None
counter = 0
done = 0
length = len(art)
done_percentage = 0

for _,i in art.iterrows():
    counter += 1
    if pd.isnull(i['authors']) == False:
        c = re.split(r'\|', i['authors'])#split
        # If several authors: we see if they all have same gender, else "groupe"
        if len(c) > 1:
            gender_list = [authors[authors['Id artist'] == int(b)]['Gender'].values[0] for b in c]
            if all("masculin" == g for g in gender_list):
                var = "masculin"
            elif all("féminin" == g for g in gender_list):
                var = "féminin"
            else:
                var = "groupe"
        # If one author: trivial
        else:
            b = c[0]
            var = authors[authors['Id artist'] == int(b)]['Gender'].values[0]

        art.at[_,'Gender'] = var
        done += 1
    
    if counter % 10000==0:
        print(f'percentage: {round(100*counter/length, 3)}%, total = {counter}, done = {done}, done percentage = {round(100*done/length, 3)}%')
        # print(i['Gender'])

percentage: 2.667%, total = 10000, done = 10000, done percentage = 2.667%
percentage: 5.333%, total = 20000, done = 20000, done percentage = 5.333%
percentage: 8.0%, total = 30000, done = 30000, done percentage = 8.0%
percentage: 10.667%, total = 40000, done = 40000, done percentage = 10.667%
percentage: 13.334%, total = 50000, done = 50000, done percentage = 13.334%
percentage: 16.0%, total = 60000, done = 60000, done percentage = 16.0%
percentage: 18.667%, total = 70000, done = 70000, done percentage = 18.667%
percentage: 21.334%, total = 80000, done = 80000, done percentage = 21.334%
percentage: 24.001%, total = 90000, done = 90000, done percentage = 24.001%
percentage: 26.667%, total = 100000, done = 100000, done percentage = 26.667%
percentage: 29.334%, total = 110000, done = 110000, done percentage = 29.334%
percentage: 32.001%, total = 120000, done = 120000, done percentage = 32.001%
percentage: 34.667%, total = 130000, done = 130000, done percentage = 34.667%
percentage: 37.334

In [85]:
authors.sample(20)

,Id artist,name,name extended,type,Birth year,Death year,Birth city,Birth state,Birth country,Death city,Death state,Death country,Gender,Nationality (original),ID artworks,average_year,artworks_creation_years
1747,9000000000078691,André Delaistre,NaN,artiste,1865.0,NaN,Paris,NaN,France,NaN,NaN,NaN,masculin,française,140000000060681,1886.0,1908
3712,140000001589535,Marcel Dupont,NaN,artiste,1907.0,1983.0,NaN,NaN,NaN,NaN,NaN,NaN,masculin,française,140000000017426,1951.0,1963
15301,9000000000087676,MALASSIS (COOPÉRATIVE DES),Groupe actif de 1970 à 1978\r\nliste des membr...,groupe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,groupe,NaN,30000000002052|30000000002137|30000000002138|3...,1972.0,1969|1969|1969|1969|1969|1969|1969|1969|1970|1...
15154,9000000000086031,Arnaud Maguet,NaN,artiste,1975.0,NaN,Toulon,Var,France,NaN,NaN,NaN,masculin,française,150000000488551|390000000000792|29000000002665...,1975.0,NaN
9641,200000000001100,Louis Holweck,NaN,artiste,1861.0,1935.0,Paris,NaN,France,NaN,NaN,NaN,masculin,française,140000000039868|140000000042684|14000000004268...,1899.0,1891|1911
31547,9000000000068600,Michel Blais,NaN,artiste,1941.0,NaN,Angers,Maine-et-Loire,France,NaN,NaN,NaN,masculin,française,490000000000149,1962.0,1984
13556,9000000000069707,Jean-François Lecourt,NaN,artiste,1958.0,NaN,Assé-le-Boisne,Sarthe,France,NaN,NaN,NaN,masculin,française,490000000000733|490000000000732|14000000006378...,1982.0,1984|1985|1985|1988|1989|1990
12327,140000000009586,Jeanne Labric,NaN,artiste,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,féminin,française,140000000057104,1915.0,1915
17140,150000001066137,"The Irresistible Force (Mixmaster Morris,Morri...",NaN,artiste,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,masculin,britannique,150000001066131,1998.0,1998
28279,230000000025622,Roland Van Meerhaeghe,NaN,artiste,1924.0,NaN,Tourcoing,Nord,France,NaN,NaN,NaN,masculin,NaN,230000000025624|230000000085546|230000000025623,1937.0,1941|1941|1943


In [8]:
art['Gender'].value_counts()

masculin         293730
féminin           47270
groupe            11602
non renseigné      6703
Name: Gender, dtype: int64

In [135]:
art['Gender'].count()/len(art) # 4,2% de NaN

0.9581671075649614

### Etape préliminaire 2 : transformer les collections mal orthographiées

In [9]:
# Transform bad collections
art.loc[art['collection'] == "Musée national d'art moderne / Centre de cr��ation industrielle", 'collection'] = "Musée national d'art moderne / Centre de création industrielle"
art.loc[art['collection'] == "Mus��e d'Art moderne et contemporain de la Ville de Strasbourg", 'collection'] = "Musée d'Art moderne et contemporain de la Ville de Strasbourg"
art.loc[art['collection'] == "Mus��e national d'art moderne / Centre de création industrielle", 'collection'] = "Musée national d'art moderne / Centre de création industrielle"
art.loc[art['collection'] == "La Piscine, Mus��e d'art et d'industrie André Diligent, Roubaix", 'collection'] = "La Piscine, Musée d'art et d'industrie André Diligent, Roubaix"
art.loc[art['collection'] == "Mus��e d'art contemporain de Lyon", 'collection'] = "Musée d'art contemporain de Lyon"
print(art['collection'].value_counts())
print(len(art['collection'].unique()))

Musée national d'art moderne / Centre de création industrielle                             105171
Centre national des arts plastiques/Fonds national d'art contemporain                       84653
Fonds municipal d'art contemporain de la Ville de Paris                                     20232
Musée Bourdelle, Paris                                                                      20191
La Piscine, Musée d'art et d'industrie André Diligent, Roubaix                              17126
Musée d'Art moderne et contemporain de la Ville de Strasbourg                               17047
Musée d'art moderne de la Ville de Paris                                                    14469
Musée d'art moderne et contemporain de Saint-Etienne Métropole                              12268
Musée national Picasso-Paris                                                                 8403
LaM, Lille Métropole, musée d'art moderne, d'art contemporain et d'art brut                  6744
Musée Tomi Ungerer, 

In [11]:
fracs = []
for museum in art['collection'].dropna().unique():
    if "frac" in museum.lower():
        fracs.append(museum)
len(fracs)

18

## Partie 1 : Proportion du nombre de femmes dans les acquisitions

In [190]:
# Function to plot the proportion of femmes in any subset of Arworks
def get_ratioF(art, subset, subset_name, path):
    art = art[art.index >= 1945]
    subset = subset[subset.index >= 1945]
    fig, ax = plt.subplots()
    # We always plot the general evolution on the background
    f = art[art['Gender']=='féminin'].groupby('acquisition_year').count()['_id']
    total = art.groupby('acquisition_year').count()['_id']
    ratio_general = f/total
    ratio_general.plot(figsize=(20,10), color=color_b, linewidth=2, ax=ax)
    # We plot the proportion in the subset of artworks of interest
    total1 = subset.groupby('acquisition_year').count()['_id']
    f1 = subset[subset['Gender']=='féminin'].groupby('acquisition_year').count()
    if len(f1) == 0:
        f1 = total1.copy()
        f1[:] = 0
    else:
        f1 = f1['_id']
    ratio = f1/total1
    ratio.plot(figsize=(20,10), color=color_f, marker='.', linewidth=2, ax=ax)
    # Then we customize the plot
    ax.set_title("Proportion d'artistes femmes dans les acquisitions, " + subset_name, fontsize=20)
    ax.set_xlabel("Année d'acquisition", fontsize=20)
    ax.set_ylabel("Proportion d'artistes femmes", fontsize=20)
    ax.set_xlim(left = 1945,right = 2017)
    ax.set_ylim(bottom = 0,top = 1)
    ax.tick_params(labelsize = 15)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(5))
    plt.grid()
    plt.savefig(path, bbox_inches='tight',format="png", transparent=True)
    plt.close()

In [39]:
def validname(value):
    """
    Normalizes string, converts to lowercase, removes non-alpha characters,
    and converts spaces to hyphens.
    """
    value = re.sub('[^\w\s-]', '', value).strip().lower()
    value = re.sub('[-\s]+', '-', value)
    return value

In [204]:
get_ratioF(art, art, "ensemble des collections", "./figures/overall.png")
get_ratioF(art, art[art['collection'].isin(fracs)], "ensemble des 18 FRAC", "./figures/fracs.png")

In [ ]:
for museum in art['collection'].dropna().unique():
    # print("Musée en cours:", museum)
    subset = art[art['collection'] == museum]
    get_ratioF(art, subset, museum, "./figures/" + validname(museum) + "/ratio_overall.png")
print('Done')

### Aparté

In [92]:
# A study on "is it a good idea to group by separable artworks" => NO
uniquegroups = art[art['type'] == 'separable'].groupby('related').first()
# uniquegroups = art[art['type'] == 'separable'].groupby('related').agg(lambda x:x.value_counts().index[0])
# matisse = art[art['related'] == '70000000007610|70000000007613|70000000007616|70000000007619|70000000007622|70000000007625|70000000007628|70000000007631|70000000007634|70000000007637|70000000007640|70000000007643|70000000007646|70000000007649|70000000007652|70000000007655|70000000007658|70000000007661|70000000007664|70000000007667|70000000007670|70000000007673|70000000007676|70000000007679|70000000007682|70000000007685|70000000007688|70000000007691|70000000007694|70000000007697|70000000007700|70000000007703|70000000007706|70000000007709|70000000007712|70000000007715|70000000007718|70000000007721|70000000007724|70000000007727|70000000007730|70000000007733|70000000007736|70000000007739|70000000007742|70000000007745|70000000007748|70000000007751|70000000007754|70000000007757|70000000007760|70000000007763|70000000007766|70000000007769|70000000007772|70000000007775|70000000007778|70000000007781|70000000007784|70000000007787|70000000007790|70000000007793|70000000007796|70000000007799|70000000007802|70000000007805|70000000007808|70000000007811|70000000007814|70000000007817|70000000007820|70000000007823|70000000007826|70000000007829|70000000007832|70000000007835|70000000007838|70000000007841|70000000007844|70000000007847|70000000007850|70000000007853|70000000007856|70000000007859|70000000007862|70000000007865|70000000007868|70000000007871|70000000007874|70000000007877|70000000007880|70000000007883|70000000007886|70000000007889|70000000007892|70000000007895|70000000007898|70000000007901|70000000007904|70000000007907|70000000007910|70000000007913|70000000007916|70000000007919|70000000007922|70000000007925|70000000007928|70000000007931|70000000007934|70000000007937|70000000007940|70000000007943|70000000007946|70000000007949|70000000007952|70000000007955|70000000007958|70000000007961|70000000007964|70000000007967|70000000007970|70000000007973|70000000007976|70000000007979|70000000007982|70000000007985|70000000007988|70000000007991|70000000007994|70000000007997|70000000008000|70000000008003|70000000008006|70000000008009|70000000008012|70000000008015|70000000008018|70000000008021|70000000008024|70000000008027|70000000008030|70000000008033|70000000008036|70000000008039|70000000008042|70000000008045|70000000008048|70000000008051|70000000008054|70000000008057|70000000008060|70000000008063|70000000008071|70000000008074|70000000008077|70000000008080|70000000008083|70000000008086|70000000008089|70000000008092|70000000008095|70000000008098|70000000008101|70000000008104|70000000008107|70000000008110|70000000008113|70000000008116|70000000008119|70000000008122|70000000008125|70000000008128|70000000008131|70000000008134|70000000008137|70000000008140|70000000008143|70000000008146|70000000008149|70000000008152|70000000008155|70000000008158|70000000008161|70000000008164|70000000008167|70000000008170|70000000008173|70000000008176|70000000008179|70000000008182|70000000008185|70000000008188|70000000008191|70000000008194|70000000008197|70000000008200|70000000008203|70000000008206|70000000008209|70000000008212|70000000008215|70000000008218|70000000008221|70000000008224|70000000008227|70000000008230|70000000008233|70000000008236|70000000008239|70000000008242|70000000008245|70000000008248|70000000008251|70000000008254|70000000008257|70000000008260|70000000008263|70000000008266|70000000008269|70000000008272|70000000008275|70000000008278|70000000008281|70000000008284|70000000008287|70000000008290|70000000008293|70000000008296|70000000008299|70000000008302|70000000008305|70000000008308|70000000008311|70000000008314|70000000008317|70000000008320|70000000008323|70000000008326|70000000008329|70000000008332|70000000008335|70000000008338|70000000008341|70000000008344|70000000008347|70000000008350|70000000008353|70000000008356|70000000008359|70000000008362|70000000008365|70000000008368|70000000008371|70000000008374|70000000008377|70000000008380|70000000008383|70000000008386|70000000008389|70000000008392|70000000008395|70000000008398|70000000008401|70000000008404|70000000008407|70000000008410|70000000008413|70000000008416|70000000008419|70000000008422|70000000008425|70000000008428|70000000008431|70000000008434|70000000008437|70000000008440|70000000008443|70000000008446|70000000008449|70000000008452|70000000008455|70000000008458|70000000008461|70000000008464|70000000008467|70000000008470|70000000008473|70000000008476|70000000008479|70000000008482|70000000008485|70000000008488|70000000008491|70000000008494|70000000008497|70000000008500|70000000008503|70000000008506|70000000008509|70000000008512|70000000008515|70000000008518|70000000008521|70000000008524|70000000008533|70000000008536|70000000008539|70000000008542|70000000008545|70000000008548|70000000008551|70000000008554|70000000008557|70000000008560|70000000008563|70000000008566|70000000008569|70000000008572|70000000008575|70000000008578|70000000008581|70000000008584|70000000008587|70000000008590|70000000008593|70000000008596|70000000008599|70000000008602|70000000008605|70000000008608|70000000008611|70000000008614|70000000008617|70000000008620|70000000008623|70000000008626|70000000008629|70000000008632|70000000008635|70000000008638|70000000008641|70000000008644|70000000008647|70000000008650|70000000008653|70000000008656|70000000008659|70000000008662|70000000008665|70000000008668|70000000008671|70000000008674|70000000008677|70000000008680|70000000008683|70000000008686|70000000008689|70000000008692|70000000008695|70000000008698|70000000008701|70000000008704|70000000008707|70000000008710|70000000008713|70000000008716|70000000008719|70000000008722|70000000008725|70000000008728|70000000008731|70000000008734|70000000008737|70000000008740|70000000008743|70000000008746|70000000008749|70000000008752|70000000008755|70000000008758|70000000008761|70000000008764|70000000008767|70000000008770|70000000008773|70000000008776|70000000008779|70000000008782|70000000008785|70000000008788|70000000008791|70000000008794|70000000008797|70000000008800|70000000024504|70000000024507|70000000024510|70000000024513|70000000024516|70000000024519|70000000024522|70000000024525|70000000024528|70000000024531|70000000024534|70000000024537|70000000024540|70000000024543|70000000024546|70000000024549|70000000024552|70000000024555|70000000024558|70000000024561|70000000024564|70000000024567|70000000024570|70000000024573|70000000024576|70000000024579|70000000024582|70000000024585|70000000024588|70000000024591|70000000024594|70000000024597|70000000024600|70000000024603|70000000024606|70000000024609']
counterex = art[art['related'] == '10000000001440|10000000001441|10000000001442|10000000001443|10000000001444|10000000001445|10000000001439|10000000001446|10000000001447|10000000001448|10000000001449|10000000001450|10000000001451|10000000001452|10000000001453|10000000001454|10000000001481']
#counterex

<h2>Partie 2 : Age moyen à l'acquisition, par genre

In [150]:
# Computing age at acquisition
art['age_at_acquisition'] = None
counter = 0
done = 0
length = len(art)
done_percentage = 0

for _,i in art.iterrows():
    counter += 1
    if pd.isnull(i['authors']) == False:
        c = re.split(r'\|', i['authors'])#split
        # If several authors: we take the average age
        if len(c) > 1:
            ages_list = [authors[authors['Id artist'] == int(b)]['Birth year'].values[0] for b in c]
            var = i['acquisition_year'] - np.nanmean(ages_list)
        # If one author: trivial
        else:
            b = c[0]
            var = i['acquisition_year'] - authors[authors['Id artist'] == int(b)]['Birth year'].values[0]

        art.at[_,'age_at_acquisition'] = var
        done += 1
    
    if counter % 100000==0:
        print(f'percentage: {round(100*counter/length, 3)}%, total = {counter}, done = {done}, done percentage = {round(100*done/length, 3)}%')

art['age_at_acquisition'] = pd.to_numeric(art['age_at_acquisition'])
art['age_at_acquisition'].count()/len(art) #18% de NaN

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: Mean of empty slice
  from ipykernel import kernelapp as app


percentage: 26.667%, total = 100000, done = 100000, done percentage = 26.667%
percentage: 53.334%, total = 200000, done = 200000, done percentage = 53.334%
percentage: 80.002%, total = 300000, done = 299996, done percentage = 80.001%


0.8275989887784273

In [148]:
art['age_at_acquisition'].sample(10)

322684    132
32736     107
184643    105
291345     52
163912    NaN
356712     37
317731    NaN
305720     81
344374     84
352737     41
Name: age_at_acquisition, dtype: object

In [203]:
# Function to plot the age at acquisition (separated between hommes and femmes)
def get_age_acquisition(art, subset, subset_name, path):
    subset = subset[subset.index >= 1945]
    fig, ax = plt.subplots()
    total = subset.groupby('acquisition_year').mean()['age_at_acquisition']
    # Femmes
    f = subset[subset['Gender']=='féminin'].groupby('acquisition_year').mean()
    if len(f) == 0:
        f = total.copy()
        f[:] = 140
    else:
        f = f['age_at_acquisition']
    # Hommes
    h = subset[subset['Gender']=='masculin'].groupby('acquisition_year').mean()
    if len(h) == 0:
        h = total.copy()
        h[:] = 140
    else:
        h = h['age_at_acquisition']
    f.plot(figsize=(20,10), color=color_f, marker='.', linewidth=2, ax=ax)
    h.plot(figsize=(20,10), color=color_h, marker='.', linewidth=2, ax=ax)
    # Then we customize the plot
    ax.set_title("Age moyen de l'artiste à l'acquisition par genre, " + subset_name, fontsize=20)
    ax.set_xlabel("Année d'acquisition", fontsize=20)
    ax.set_ylabel("Age moyen de l'artiste", fontsize=20)
    ax.set_xlim(left = 1945,right = 2017)
    ax.set_ylim(bottom = 0,top = 140)
    ax.tick_params(labelsize = 15)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(5))
    plt.grid()
    plt.savefig(path, bbox_inches='tight',format="png", transparent=True)
    plt.close()

In [201]:
get_age_acquisition(art, art, "ensemble des collections", "./figures/age_overall.png")
get_age_acquisition(art, art[art['collection'].isin(fracs)], "ensemble des 18 FRAC", "./figures/age_fracs.png")

In [202]:
for museum in art['collection'].dropna().unique():
    # print("Musée en cours:", museum)
    subset = art[art['collection'] == museum]
    get_age_acquisition(art, subset, museum, "./figures/" + validname(museum) + "/age_acquisition.png")
print('Done')

Done


## Partie 3 : ratio d'artiste Français dans les acquisitions, par genre

In [ ]:
authors['nationality_clean'].value_counts()

In [ ]:
art['nationality'] = None
counter = 0
done = 0
length = len(art)
done_percentage = 0

for _,i in art.iterrows():
    counter += 1
    if pd.isnull(i['authors']) == False:
        c = re.split(r'\|', i['authors'])#split
        # If several authors: we see if they all have same gender, else "groupe"
        if len(c) > 1:
            nat_list = [authors[authors['Id artist'] == int(b)]['nationality_clean'].values[0] for b in c]
            if any("français" in g for g in nat_list):
                var = "français"
            else:
                var = "groupe"
        # If one author: trivial
        else:
            b = c[0]
            var = authors[authors['Id artist'] == int(b)]['nationality_clean'].values[0]

        art.at[_,'nationality'] = var
        done += 1
    
    if counter % 10000==0:
        print(f'percentage: {round(100*counter/length, 3)}%, total = {counter}, done = {done}, done percentage = {round(100*done/length, 3)}%')